Read dataset

In [8]:
import pandas as pd
from datetime import datetime
df1 = pd.concat(pd.read_excel('PV_firstRealease.xlsx', sheet_name=None), ignore_index=True)
df1.head()

,Hour,Starting minute (inclussive),Ending minute (exclussive),Generated power,TemperatureC,DewpointC,PressurehPa,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,dailyrainMM,SolarRadiationWatts_m2
0,0,0,5,0,0,0,0,0,0,0,0,0,0,0
1,0,5,10,0,0,0,0,0,0,0,0,0,0,0
2,0,10,15,0,11,8,1021,114,1,17,83,0,0,0
3,0,15,20,0,11,8,1021,134,12,17,83,0,0,0
4,0,20,25,0,11,8,1021,128,8,17,83,0,0,0


Remove outliers

In [31]:
def remove_dataset_ouliers(dataset):
        list_outliers_index=[]
        for c in dataset:
            if c!="DateTime":
                list_outliers_index.extend(column_outliers(dataset, c))
        cleaned_dataset=remove_outliers(dataset, list_outliers_index)
        return cleaned_dataset

def column_outliers(df,c):
    q1=df[c].quantile(0.25)
    q3=df[c].quantile(0.75)
    iqr=q3-q1
    lower_bound=q1-4*iqr
    upper_bound=q3+4*iqr
    o = df.index[(df[c]<lower_bound) | (df[c]>upper_bound)]
    return o

def remove_outliers(df, outliers):
    outliers=sorted(set(outliers))
    df=df.drop(outliers)
    return df

Fix dataset structure 

In [32]:
df=df1
df["DateTime"] = df['Hour'].astype(str) +":"+ df["Starting minute (inclussive)"].astype(str)
df=df.drop(columns='Hour')
df=df.drop(columns='Ending minute (exclussive)')
df=df.drop(columns='Starting minute (inclussive)')
# set the initial date to '1989-12-31' and convert the time column to a datetime column
date='1989-12-31'
# convert the datetime strings to datetime objects
datetime_object= pd.to_datetime(date, format='%Y-%m-%d').date()
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%H:%M')
startDate=df['DateTime'][0].time()
firstTime=True
dates=df['DateTime']
for index in df.index:
    if dates[index].time()==startDate:
        datetime_object=datetime_object+ pd.Timedelta(days=1)
        date_entry=df['DateTime'][index].time()
        new_datetime_entry=str(datetime_object)+" "+str(date_entry)
        new_datetime_entry= pd.to_datetime(new_datetime_entry, format='%Y-%m-%d %H:%M')
        df.loc[index,"DateTime"]=new_datetime_entry
    else:
        date_entry=df['DateTime'][index].time()
        new_datetime_entry=str(datetime_object)+" "+str(date_entry)
        new_datetime_entry= pd.to_datetime(new_datetime_entry, format='%Y-%m-%d %H:%M')
        df.loc[index,"DateTime"]=new_datetime_entry
first_column=df.pop("DateTime")
df.insert(0, "DateTime", first_column)
df = df.rename_axis('index')
df.set_index("DateTime",append=True, inplace=True)
df=df.resample('15min',level='DateTime').mean()
df.reset_index(inplace=True)
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%H:%M').dt.time
df.head()

,DateTime,Generated power,TemperatureC,DewpointC,PressurehPa,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,dailyrainMM,SolarRadiationWatts_m2
0,00:00:00,0.0,3.666667,2.666667,340.333333,38.000000,0.333333,5.666667,27.666667,0.0,0.0,0.0
1,00:15:00,0.0,11.000000,8.000000,1021.000000,128.333333,8.666667,17.000000,83.000000,0.0,0.0,0.0
2,00:30:00,0.0,11.000000,8.000000,1021.000000,133.333333,1.333333,19.000000,83.000000,0.0,0.0,0.0
3,00:45:00,0.0,11.000000,8.000000,1021.000000,127.666667,3.000000,12.333333,82.333333,0.0,0.0,0.0
4,01:00:00,0.0,11.000000,8.000000,1021.000000,128.000000,1.333333,10.000000,81.000000,0.0,0.0,0.0


Call the function to remove outliers

In [34]:
df_no_out=remove_dataset_ouliers(df)
print('Before', len(df), 'After', len(df_no_out), 'Difference',(len(df)-len(df_no_out)))

Before 105120 After 70647 Difference 34473


In [ ]:
df1